<a href="https://colab.research.google.com/github/awhitehouse1/project_voting/blob/main/cleaning_votes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

vdf = pd.read_csv('./data/voting_VA.csv') # Load historical voting data
#cdf = pd.read_csv('./data/county_adjacencies.csv') # Load county adjacency data and population data


In [4]:
vdf.head()


,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL


In [3]:
print(vdf.shape)
print(vdf.dtypes) # all of the variables have the correct types

(3736, 13)
Unnamed: 0         int64
year               int64
state             object
state_po          object
county_name       object
county_fips        int64
office            object
candidate         object
party             object
candidatevotes     int64
totalvotes         int64
version            int64
mode              object
dtype: object


In [5]:
print(vdf['party'].value_counts()) # get the number of rows for each party

print(vdf.groupby(["party"]).candidatevotes.sum())
# non democrat and republican votes make up only 2.4% of all votes in the dataset

party
DEMOCRAT       1068
REPUBLICAN     1068
OTHER          1068
LIBERTARIAN     398
GREEN           134
Name: count, dtype: int64
party
DEMOCRAT       10997557
GREEN             59373
LIBERTARIAN       64761
OTHER            406335
REPUBLICAN     10433236
Name: candidatevotes, dtype: int64


Votes for third parties (libertarian, green, and other) make up only about 2.4% of all votes in the dataset. Because votes for republican and democrat parties make up the majority of votes, all third party votes will be dropped.

In [ ]:
clean_vdf = vdf.drop(vdf[vdf['party'] == 'OTHER'].index)
clean_vdf = clean_vdf.drop(clean_vdf[clean_vdf['party'] == 'LIBERTARIAN'].index)
clean_vdf = clean_vdf.drop(clean_vdf[clean_vdf['party'] == 'GREEN'].index)
clean_vdf